In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

#from spatial_transformer import SpatialTransformer
np.random.seed(123)  # for reproducibility

import matplotlib.pyplot as plt
%matplotlib inline

from scipy import misc
import glob

def load_train(img_path):
    imageArr = np.zeros((1, 32, 32, 3))
    iCount = 1
    for image_path in glob.glob(img_path):#("test/*.png"):
        image = misc.imread(image_path)
        image = image.reshape(1,32,32,3)
        imageArr = np.concatenate((imageArr, image), axis=0)
        # print (image.shape)
        # print (image.dtype)
        #(32, 32, 3)   uint8
        if iCount%500 == 0:
            print ("500new loaded:",imageArr.shape)
        iCount += 1
        # print (imageArr.shape)
    print(imageArr.shape)
    imageArr = np.delete(imageArr, imageArr[0], axis=0)
    print ("images imported: ",imageArr.shape)
    return imageArr

def load_labels(file_path):
    mnist_df = pd.read_csv("train.csv", index_col=None)
    print('train label sizes:', mnist_df.shape)
    labels = mnist_df.iloc[0:mnist_df.shape[0],0:1]
    labels_data = labels.values
    print('labels:',labels.shape, labels.shape[-1])
    return labels_data

In [ ]:
cache_path = os.path.join('cache', 'train.dat')
if not os.path.isfile(cache_path):
    # train_data, train_target = load_train()
    X_train = load_train('train/*.png')
    y_train = load_labels('train.csv')
    #X_train, y_train = load_train_dic('onlyletter/')
    # load_train('trimeddataset/g1')
    print(X_train.shape, y_train.shape)
    print(y_train)
    cache_data((X_train, y_train), cache_path)
    print("cache_data")
else:
    print('Restore train from cache!')
    (X_train, y_train) = restore_data(cache_path)
    print(X_train.shape, y_train.shape)

500new loaded: (501, 32, 32, 3)
500new loaded: (1001, 32, 32, 3)
500new loaded: (1501, 32, 32, 3)
500new loaded: (2001, 32, 32, 3)
500new loaded: (2501, 32, 32, 3)
500new loaded: (3001, 32, 32, 3)
500new loaded: (3501, 32, 32, 3)
500new loaded: (4001, 32, 32, 3)
500new loaded: (4501, 32, 32, 3)
500new loaded: (5001, 32, 32, 3)
500new loaded: (5501, 32, 32, 3)
500new loaded: (6001, 32, 32, 3)
500new loaded: (6501, 32, 32, 3)
500new loaded: (7001, 32, 32, 3)
500new loaded: (7501, 32, 32, 3)
500new loaded: (8001, 32, 32, 3)
500new loaded: (8501, 32, 32, 3)
500new loaded: (9001, 32, 32, 3)
500new loaded: (9501, 32, 32, 3)
500new loaded: (10001, 32, 32, 3)
500new loaded: (10501, 32, 32, 3)
500new loaded: (11001, 32, 32, 3)
500new loaded: (11501, 32, 32, 3)
500new loaded: (12001, 32, 32, 3)
500new loaded: (12501, 32, 32, 3)
500new loaded: (13001, 32, 32, 3)
500new loaded: (13501, 32, 32, 3)
500new loaded: (14001, 32, 32, 3)
500new loaded: (14501, 32, 32, 3)
500new loaded: (15001, 32, 32, 3)


In [ ]:
cache_path = os.path.join('cache', 'test.dat')
if not os.path.isfile(cache_path):
    # train_data, train_target = load_train()
    X_test = load_train('test/*.png')
    #y_test = load_labels('train.csv')
    #X_train, y_train = load_train_dic('onlyletter/')
    # load_train('trimeddataset/g1')
    print(X_test.shape)
    cache_data(X_test, cache_path)
    print("cache_data for test")
else:
    print('Restore test from cache!')
    X_test = restore_data(cache_path)
    print(X_test.shape)

In [ ]:
input_shape = (32,32,3)
batch_size = 256
num_classes = 10
epochs = 400
data_augmentation = True

# for i in range(5, 9):
#     plt.subplot(330 + (i+1))
#     plt.imshow(imgTrain[i], cmap=plt.get_cmap('gray'))
#     plt.title(labels_data[i])

In [286]:
from keras.callbacks import EarlyStopping,CSVLogger,ModelCheckpoint

# optimizers = ['adam','rmsprop']
# activations = ['relu','PReLU','LeakyReLU']
# activations_ps = ['relu']
# dropouts = [0.6,0.5,0,0.3]
# # padding = ['same','valid']
# cost_fun = [loss_max,'mse']

callbacks = [EarlyStopping(monitor='val_loss',patience=3,verbose=1)]

In [287]:
from keras.datasets import cifar10

(x_trainK, y_trainK), (x_testK, y_testK) = cifar10.load_data()
# x_trainK = x_trainK.reshape(x_trainK.shape[0], x_trainK.shape[1], x_trainK.shape[2], 3)
# x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
# y_train = y_train.reshape(y_train.shape[0], 1)
# y_test = y_test.reshape(y_test.shape[0], 1)
print (x_trainK.shape, y_trainK.shape, x_testK.shape, y_testK.shape)
print (X_train.shape, y_train.shape)
X_train = np.concatenate((x_trainK, X_train), axis=0)
X_train = np.concatenate((x_testK, X_train), axis=0)
y_train = np.concatenate((y_trainK, y_train), axis=0)
y_train = np.concatenate((y_testK, y_train), axis=0)

y_train = np_utils.to_categorical(y_train, num_classes)

#np.random.shuffle(images_data)


(60000, 28, 28, 1) (60000, 1) (10000, 28, 28, 1) (10000, 1)
(42000, 28, 28, 1) (42000, 1)


In [289]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

def selu(x):
    import tensorflow as tf
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale*tf.where(x >= 0.0, x, alpha*tf.nn.elu(x))

# initial weights
b = np.zeros((2, 3), dtype='float32')
b[0, 0] = 1
b[1, 1] = 1
W = np.zeros((50, 6), dtype='float32')
weights = [W, b.flatten()]

locnet = Sequential()
locnet.add(MaxPooling2D(pool_size=(2,2), input_shape=input_shape))
locnet.add(Conv2D(20, (5, 5)))
locnet.add(MaxPooling2D(pool_size=(2,2)))
locnet.add(Conv2D(20, (5, 5)))

locnet.add(Flatten())
locnet.add(Dense(50))
locnet.add(Activation('relu'))
locnet.add(Dense(6, weights=weights))
#locnet.add(Activation('sigmoid'))

In [290]:
def norm_input(x):
    mean = X_train.mean().astype(np.float32)
    std = X_train.std().astype(np.float32)
    return (x - mean) / std

In [292]:
def my_deeper_cnn_model():
    model = Sequential()
    #model.add(SpatialTransformer(localization_net=locnet, output_size=(30, 30), input_shape=input_shape))
    #model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(1,28,28)))
    
    model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(BatchNormalization()) # customized 
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
#     #customized op
#     from keras import optimizers
#     ad10e4 = optimizers.Adam(lr=0.0005)
#     #commpile modelPython
#     model.compile(optimizer=ad10e4,
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

#save the uncoded labels
labels_data = y_train
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# print(encoded_Y.shape, encoded_Y) 
# [0,1,2,3...,34,35]
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y.shape, dummy_y)
# [1,0,0....0][0,1,0....,0][0,0,1,...,0]

y_train = dummy_y
print(y_train.shape)

In [294]:
model = my_deeper_cnn_model()

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.01,
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))








#Fit keras Model
# model.fit(X_train, Y_train,
#           batch_size=256, epochs=30, verbose=1, callbacks=callbacks,validation_data=(X_test,Y_test)) #instead of 15epoch
#model.fit(images_data, labels_data,
#          batch_size=256, epochs=18, verbose=1, validation_split=0.0001) #instead of 15epoch
# callbacks=callbacks,
# Epoch 1/10
# 7744/60000 [==>...........................] - ETA: 96s - loss: 0.5806 - acc: 0.8164

#score = model.evaluate(X_test, Y_test, verbose=0)
#print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
print('debuging')
#
# from keras.models import model_from_json
#
# # split into input (X) and output (Y) variables
# # X = dataset[:,0:8]
# # Y = dataset[:,8]
# # # create model
# # model = Sequential()
# # model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
# # model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
# # model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# # # Compile model
# # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# # # Fit the model
# # model.fit(X, Y, epochs=150, batch_size=10, verbose=0)
# # # evaluate the model
# # scores = model.evaluate(X, Y, verbose=0)
# # print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#
# # serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")
#
# # later...
#
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
#
# # evaluate loaded model on test data
# loaded_model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# score = loaded_model.evaluate(X_test, Y_test, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

# from keras.models import load_model
#
# model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
# #del model  # deletes the existing model
#
# # returns a compiled model
# # identical to the previous one
# model_L = load_model('my_model.h5')
# model_L.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# score = model_L.evaluate(X_test, Y_test, verbose=0)
# print("%s: %.2f%%" % (model_L.metrics_names[1], score[1]*100))


(None, 14, 14, 64)
(None, 2, 2, 256)
Train on 111988 samples, validate on 12 samples
Epoch 1/18
111988/111988 [==============================] - 603s - loss: 0.0353 - acc: 0.9881 - val_loss: 0.0129 - val_acc: 1.0000
Epoch 2/18
111988/111988 [==============================] - 618s - loss: 0.0097 - acc: 0.9971 - val_loss: 1.2737e-05 - val_acc: 1.0000
Epoch 3/18
111988/111988 [==============================] - 501s - loss: 0.0069 - acc: 0.9979 - val_loss: 1.1313e-06 - val_acc: 1.0000
Epoch 4/18
111988/111988 [==============================] - 495s - loss: 0.0062 - acc: 0.9982 - val_loss: 1.9776e-06 - val_acc: 1.0000
Epoch 5/18
111988/111988 [==============================] - 589s - loss: 0.0049 - acc: 0.9986 - val_loss: 1.1550e-07 - val_acc: 1.0000
Epoch 6/18
111988/111988 [==============================] - 536s - loss: 0.0046 - acc: 0.9986 - val_loss: 3.8074e-06 - val_acc: 1.0000
Epoch 7/18
111988/111988 [==============================] - 494s - loss: 0.0041 - acc: 0.9988 - val_loss: 2.4

In [ ]:
# rst = model_L.predict_classes(images_datats, verbose=0)
# print(rst.shape)

rst = model.predict_classes(X_test, verbose=1)
print(rst.shape)
rststr = encoder.inverse_transform(rst)
rst_df = pd.DataFrame({"id": list(range(1,len(rst)+1)),
                         "label": rststr})
rst_df.to_csv('rstpreditct0723.csv', index=False, header=True)

In [ ]:
from keras.models import load_model

model.save_weights("model.h5")
print("Saved model to disk")

# del model  # deletes the existing model

model_L = load_model('my_model.h5')
model_L.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
score = model_L.evaluate(X_test, Y_test, verbose=0)
print("%s: %.2f%%" % (model_L.metrics_names[1], score[1]*100))

In [ ]:
# use pretrained model

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)
    train = model.predict_generator(train_generator, train_generator.nb_sample)
    test = model.predict_generator(test_generator, test_generator.nb_sample)
    with h5py.File("gap_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(Xception, (299, 299), xception.preprocess_input)

import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)
X_train = []
X_test = []
for filename in ["gap_ResNet50.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

from keras.models import *
from keras.layers import *
np.random.seed(2017)
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

digraph G{
    node [shape=record]
    a[label="ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}"]
    b[label="InceptionV3|{input:|output:}|{(299, 299, 3)|(2048)}"]
    c[label="Xception|{input:|output:}|{(299, 299, 3)|(2048)}"]
    Merge[label="Merge|{input:|output:}|{(3, 2048)|(6144)}"]
    Dropout[label="Dropout|Rate:|0.5"]
    Output[label="Output|{input:|output:}|{(6144)|(1)}"]
    Image -> a -> Merge
    Image -> b -> Merge
    Image -> c -> Merge
    Merge -> Dropout -> Output
}

model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

rst = model.predict_classes(X_test, verbose=1)
print(rst.shape)
rststr = encoder.inverse_transform(rst)
rst_df = pd.DataFrame({"id": list(range(1,len(rst)+1)),
                         "label": rststr})
rst_df.to_csv('rstpreditct0723ooo.csv', index=False, header=True)